In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
#symbols = ["H","H","H","H"]
symbols = ["Li", "H"]
r_bohr = 3.0*1.8897259886  #5.0 Angstrom in Bohr 
#coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
coordinates = np.array([[0.0,0.0, 0*r_bohr], [0.0, 0.0, 1*r_bohr]])
# Calculation of the Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf")


electrons = 4
orbitals = 12
# Calculation of the excitation operators
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=False)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)  #HF state : [1,1,1,1,0,0,0,0] - [alpha, beta, alpha, beta, alpha, beta, alpha, beta]
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
print('HF state is', circuit(hf_state, electrons, qubits, H))

print('The original hamiltonian is', H)
#Putting all the functions here
#1.1091571486954503

Singles are [[0, 4], [0, 6], [0, 8], [0, 10], [1, 5], [1, 7], [1, 9], [1, 11], [2, 4], [2, 6], [2, 8], [2, 10], [3, 5], [3, 7], [3, 9], [3, 11]]
Doubles are [[0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 4, 9], [0, 1, 4, 11], [0, 1, 5, 6], [0, 1, 5, 8], [0, 1, 5, 10], [0, 1, 6, 7], [0, 1, 6, 9], [0, 1, 6, 11], [0, 1, 7, 8], [0, 1, 7, 10], [0, 1, 8, 9], [0, 1, 8, 11], [0, 1, 9, 10], [0, 1, 10, 11], [0, 2, 4, 6], [0, 2, 4, 8], [0, 2, 4, 10], [0, 2, 6, 8], [0, 2, 6, 10], [0, 2, 8, 10], [0, 3, 4, 5], [0, 3, 4, 7], [0, 3, 4, 9], [0, 3, 4, 11], [0, 3, 5, 6], [0, 3, 5, 8], [0, 3, 5, 10], [0, 3, 6, 7], [0, 3, 6, 9], [0, 3, 6, 11], [0, 3, 7, 8], [0, 3, 7, 10], [0, 3, 8, 9], [0, 3, 8, 11], [0, 3, 9, 10], [0, 3, 10, 11], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 4, 9], [1, 2, 4, 11], [1, 2, 5, 6], [1, 2, 5, 8], [1, 2, 5, 10], [1, 2, 6, 7], [1, 2, 6, 9], [1, 2, 6, 11], [1, 2, 7, 8], [1, 2, 7, 10], [1, 2, 8, 9], [1, 2, 8, 11], [1, 2, 9, 10], [1, 2, 10, 11], [1, 3, 5, 7], [1, 3, 5, 9], [1, 3, 5, 11], [1, 3, 7, 9], [1

In [2]:
# Commutator calculation for HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev,diff_method='backprop')
def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)   #Calculating the commutator
    #res = (qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Commutator calculation for other states except HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev,diff_method='backprop')
def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w) #Calculating the commutator
    return qml.expval(res)

# Energy calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev, diff_method='backprop')
def ash(params, ash_excitation, hf_state, qubits, H):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #qml.DoubleExcitation(params[i], wires = ash_excitation[i])  #Applying the double excitation and their parameters
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            #qml.SingleExcitation(params[i], wires = ash_excitation[i]) #Applying the single excitation and their parameters
            qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])

    return qml.expval(H)  #Calculating the expectation value of the Hamiltonian

# Calculation of New state, same as the above function but with the state return

dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev, diff_method='backprop')
def new_state(hf_state, ash_excitation, fparams, params):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', params[i])
            #qml.DoubleExcitation(params[i], wires = ash_excitation[i])
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])

        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', params[i])
            #qml.SingleExcitation(params[i], wires = ash_excitation[i])
            qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])

            

    return qml.state()

In [3]:

electrons = 4
orbitals = 12
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)  #Optimizer

print('The Hamiltonian is ', H)
adit = 5
fparams = []
excitations= []
operator_pool = (singles) + (doubles)  #Operator pool - Singles and Doubles
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)  #Adapt iteration
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)   #Current excitation operator - fermionic one
        w = qml.fermi.jordan_wigner(i)  #JW transformation
        if np.array_equal(k, hf_state): # If the current state is the HF state
            print('Print, if this is activated - HF state')
            current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
        else:
            current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="backprop") #Cost function definition, gradient calculated automatically and method is backprop
    print('Going to do energy calculation')
    for n in range(150):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
    fparams.append(params)
    print('Updated params are', params)
    print('Updated excitation are', ash_excitation)
    
    # New state generation - With the updated parameters
    ostate = new_state(hf_state, ash_excitation, fparams, params)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,fparams,params))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


The Hamiltonian is  -4.364494775847591 * I(0) + 1.0008882262317973 * Z(0) + 0.013681965882420419 * (Y(0) @ Z(1) @ Y(2)) + 0.013681965882420419 * (X(0) @ Z(1) @ X(2)) + 0.022532898726150788 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.022532898726150788 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.0033658793417773317 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10)) + 0.0033658793417773317 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10)) + -0.12626529300361555 * Z(2) + 0.06495628783644002 * (Z(0) @ Z(2)) + 0.009749098493368009 * (Y(2) @ Z(3) @ Y(4)) + 0.009749098493368009 * (X(2) @ Z(3) @ X(4)) + -0.013382297746973035 * (Z(0) @ Y(2) @ Z(3) @ Y(4)) + -0.013382297746973035 * (Z(0) @ X(2) @ Z(3) @ X(4)) + 0.02294762903118147 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10)) + 0.02294762903118147 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10)) + -0.021044465055204752 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ 

/Users/cwq/CodingwithQiskit/miniconda3/envs/cwq/lib/python3.12/site-packages/autoray/autoray.py:1127: ComplexWarning: Casting complex values to real discards the imaginary part
  return x.astype(dtype, **kwargs)


The expectation value of a⁺(0) a(4) is 3.169567821745267e-09
The current excitation operator is a⁺(0) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(6) is 0.0
The current excitation operator is a⁺(0) a(8)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(8) is 0.0
The current excitation operator is a⁺(0) a(10)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(10) is 3.893325398185432e-09
The current excitation operator is a⁺(1) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(5) is 3.169567759295222e-09
The current excitation operator is a⁺(1) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(7) is 0.0
The current excitation operator is a⁺(1) a(9)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(9) is 0.0
The current excitation operator is a⁺(1) a(11)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(11) is 3.89332

In [7]:
-7.79580014 - (-7.795659050519)

-7.79604106257554 - (-7.795659050519)

-0.000382012056539871

In [ ]:
print(qml.specs(cost_fn)(params, ash_excitation, hf_state, qubits, H))
ash_excitation

## Implementing scipy Minimize

In [6]:
from scipy.optimize import minimize



dev = qml.device('default.qubit', wires=12)
ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [2, 6]]
#ash_excitation =[[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4]]
ash_excitation = [[2, 3, 10, 11], [2, 3, 4, 11], [2, 3, 5, 10], [2, 3, 4, 5], [2, 4]]


@qml.qnode(dev)
def circuit(ash_excitation, hf_state, params, H):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            #qml.DoubleExcitation(params[i], wires = ash_excitation[i])
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            #print('Single Exc going in is', ash_excitation[i])
            #qml.SingleExcitation(params[i], wires = ash_excitation[i])
            qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
    return qml.expval(H)

#def grad_callback(params, jac):
    # Calculate the norm of the gradient (jac)
#    grad_norm = np.linalg.norm(jac)
#    print(f"Current gradient norm: {grad_norm}")

def cost(params):
    energy = circuit(ash_excitation, hf_state, params, H)
    return energy

def callback(params):
    print(f"Current parameters: {params}")
    print(f"Current cost: {cost(params)}\n")

params = np.array([0.99, 0.012, 0.05, 0.5,10], requires_grad=True)

#print(minimize(cost, params, method='BFGS'))

result = minimize(cost, params, method='BFGS', callback=callback)

print("Final parameters:", result.x)
print("Final cost:", result.fun)

Current parameters: [ 0.4265868  -0.39915731 -0.12719358  0.18220294  9.64615941]
Current cost: -7.66713634176465

Current parameters: [-0.05283213 -0.81745835 -0.29508378 -0.19078116  9.3309397 ]
Current cost: -7.685817527690314

Current parameters: [ 0.13318527 -0.71005491 -0.24774096 -0.14493151  9.43288246]
Current cost: -7.697780814122695

Current parameters: [ 0.24982035 -0.70135212 -0.23929698 -0.23550615  9.48358838]
Current cost: -7.706830138991178

Current parameters: [ 0.3799984  -0.7611737  -0.25944029 -0.51008984  9.53736521]
Current cost: -7.720577736743858

Current parameters: [ 0.43490353 -0.87505096 -0.31236278 -0.91358737  9.58236418]
Current cost: -7.7299090428667325

Current parameters: [ 0.37219149 -0.89806889 -0.33433172 -1.01943125  9.59206132]
Current cost: -7.7336898293306975

Current parameters: [ 0.2453114  -0.8842153  -0.35884779 -1.15349821  9.63928966]
Current cost: -7.73954432282737

Current parameters: [ 0.08406565 -0.79501075 -0.3692383  -1.29077268  9.

## Jax interface - Optax

In [ ]:
import jax
from jax import numpy as jnp
import pennylane as qml
import optax
import jax.numpy as jnp
from pennylane import qchem
from pennylane import numpy as np

jax.config.update("jax_enable_x64", True)


X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "pyscf")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
qubits = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")



dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))
    #[qml.PauliX(i) for i in jnp.nonzero(hf_state)[0]] #HF state

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#dev = qml.device("default.qubit", wires=qubits)

#@jax.jit
#@qml.qnode(dev, interface="jax")
#def circuit(param):
#    qml.RX(param, wires=0)
##    qml.CNOT(wires=[0, 1])
#    return qml.expval(H)

#ash_excitation = [[2,6],[1, 2, 4, 7],[0,1,4,5],[0, 3, 5, 6],[2, 3, 6, 7]]
#ash_excitation =  [[3, 7], [0, 4], [1, 2, 4, 7], [0, 1, 4, 5], [0, 3, 5, 6], [2, 3, 6, 7]]
ash_excitation =  [[2,3,6,7], [0,3,5,6], [0,1,4,5], [1,2,4,7], [2,6]]

optimizer = optax.adam(learning_rate=0.1)
#optimizer = optax.lbfgs()
params = jnp.zeros(len(ash_excitation), dtype=jnp.float32)
opt_state = optimizer.init(params)
print('The parameters that are going in ', len(params))

dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
#@jax.jit
@qml.qnode(dev, interface="jax")
def circuit(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            #Exchanged the order of wires 1 and wires 2
            #qml.FermionicDoubleExcitation(weight = params[i], wires2=excitations[2:][::-1], wires1=excitations[:2][::-1])
            qml.FermionicDoubleExcitation(weight = params[i], wires1=excitations[2:][::-1], wires2=excitations[:2][::-1])
        else:
            #print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations)
    return qml.expval(H)

def cost_fn(params, ash_excitation, hf_state, qubits, H):
    # Compute the energy
    energy = circuit(params, ash_excitation, hf_state, qubits, H)
    return energy

def minimize_function(params):
    energy= cost_fn(params, ash_excitation, hf_state, qubits, H)
    return energy

gradient_tolerance = 1e-8 
gradient_max_threshold = 1e-8
for i in range(250):
    #cost, grad_circuit = jax.value_and_grad(circuit)(params)
    cost, grad_circuit = jax.value_and_grad(minimize_function)(params)
        # Compute the norm of the gradient
    grad_norm = jnp.linalg.norm(grad_circuit)
    grad_max = jnp.max(jnp.abs(grad_circuit))


    # Print the current gradient norm (optional for debugging)
    print(f"Step {i}, cost: {cost}, grad_norm: {grad_norm}, grad_max: {grad_max}")

    # Check if the gradient norm is below the tolerance
    if grad_norm < gradient_tolerance:
        print(f"Convergence achieved at step {i}. Gradient norm {grad_norm} is below tolerance.")
        break
    if grad_max < gradient_max_threshold:
        print(f"Convergence achieved at step {i}. Maximum gradient {grad_max} is below the threshold.")
        break 
    updates, opt_state = optimizer.update(grad_circuit, opt_state)
    params = optax.apply_updates(params, updates)
    #print(f"step {i}, cost {cost}")
print('Final energy is', cost)

## Unitary operator

In [ ]:
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=False)
print(singles)


In [ ]:
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev,diff_method='backprop')
def commutator_new(H,comm, hf_state):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
    qml.BasisState(hf_state, wires=range(qubits))
    res = qml.commutator(H, comm)   #Calculating the commutator
    return qml.expval(res)

In [ ]:
comm = []
sing = []
#[FermiWord({(0, 0): '+', (1, 4): '-'})]
print(singles)
#Changed the index - singles [r,p] = cp+ cr. So [0,4] should generate c4+ c0 
fermionic_singles = [qml.fermi.FermiWord({(0, x[1]): "+", (1, x[0]): "-"}) for x in singles]
print(fermionic_singles)

#Below code is represented as [4,0]
fermionic_singles1 = [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
print(fermionic_singles1)
max_value = float('-inf')
max_operator = None

for i , j in zip(fermionic_singles, fermionic_singles1):
    print('The operator going in', i)
    wi = qml.jordan_wigner(i)
    print('The operator going in', j)
    wj = qml.jordan_wigner(j)
    comm = qml.commutator(wj, wi)
    print('The current excitation operator output is',comm)
    current_value = (commutator_new(H, comm, hf_state))
    sing.append(comm)
    print(f'The expectation value of {i} is', current_value)

    if current_value > max_value:
        max_value = current_value
        max_operator = i

print(f"The highest operator value is {max_value} for operator {max_operator}")
